In [ ]:
# Install Selenium, Chromium, and Driver
!pip install pandas

# Update system packages and install Chromium and Chromedriver
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver

# Install required Python libraries
!pip install selenium
!pip install openpyxl
!pip install webdriver-manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import logging


In [1]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Set up the WebDriver with enhanced stealth
def setup_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_page_load_timeout(30)  # Page load timeout
    driver.set_script_timeout(30)  # Script execution timeout
        # Enable CDP for capturing network cookies
    driver.execute_cdp_cmd("Network.enable", {})
    return driver

# Capture cookies from network traffic using CDP
def capture_network_cookies(driver):
    try:
        # Get all cookies set during network responses
        network_cookies = driver.execute_cdp_cmd("Network.getAllCookies", {})["cookies"]
        if network_cookies:
            #logging.info(f"Cookies retrieved via Network CDP: {network_cookies}")
            return [{"name": c["name"], "value": c["value"]} for c in network_cookies]
        logging.warning("No cookies available via Network CDP.")
        return []
    except Exception as e:
        logging.error(f"Error capturing network cookies: {e}")
        return []

# Execute JavaScript to extract cookies
def analyze_cookies_with_js(driver):
    try:
        # Retrieve cookies from different methods
        selenium_cookies = driver.get_cookies()  # Selenium cookies
        js_script = """
        try {
            return document.cookie.split(';').map(cookie => {
                const [name, value] = cookie.split('=');
                return { name: name.trim(), value: value };
            });
        } catch (e) {
            return [];
        }
        """
        js_cookies = driver.execute_script(js_script)  # JavaScript cookies
        cdp_cookies = capture_network_cookies(driver)  # CDP cookies

        # Combine all cookies into a single list
        combined_cookies = selenium_cookies + js_cookies + cdp_cookies

        # Deduplicate cookies using a dictionary
        unique_cookies = {}
        for cookie in combined_cookies:
            # Use (name, domain, path) as a unique key
            key = (cookie.get("name"), cookie.get("domain"), cookie.get("path"))
            unique_cookies[key] = cookie

        # Return the deduplicated list of cookies
        return list(unique_cookies.values())

    except Exception as e:
        logging.error(f"Error accessing cookies: {e}")
        return []




# Enhanced detection for cookie compliance
def evaluate_cookie_consent(driver, url):
    try:
        driver.get(url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        initial_cookies = analyze_cookies_with_js(driver)
        initial_cookies_length = len(initial_cookies)  # Length of cookies before accepting

        banner_xpath = (
            "//*[contains(@id, 'cookie') or contains(@id, 'Cookie') "
            "or contains(@class, 'consent') or contains(@class, 'Consent') "
            "or contains(text(), 'cookie') or contains(text(), 'Cookie') "
            "or contains(text(), 'consent') or contains(text(), 'Consent')]"
        )
        cookie_banner = driver.find_elements(By.XPATH, banner_xpath)

        accept_button = driver.find_elements(By.XPATH, 
            "//*[contains(text(), 'Accept') or contains(text(), 'accept') or contains(text(), 'Allow') or contains(text(), 'allow') or contains(text(), 'Allow Cookies') or contains(text(), 'allow cookies')]"
        )
        reject_button = driver.find_elements(By.XPATH, 
            "//*[contains(text(), 'Reject') or contains(text(), 'reject') or contains(text(), 'Decline') or contains(text(), 'decline')]"
        )

        if accept_button:
            try:
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable(accept_button[0])).click()
            except Exception:
                driver.execute_script("arguments[0].click();", accept_button[0])
            time.sleep(2)

        final_cookies = analyze_cookies_with_js(driver)
        final_cookies_length = len(final_cookies)  # Length of cookies after accepting

        if cookie_banner and accept_button and reject_button:
            if len(initial_cookies) == 0 and len(final_cookies) > 0:
                return "Compliant - Consent Required Before Setting Cookies", initial_cookies_length, final_cookies_length
            elif len(initial_cookies) > 0:
                return "Non-Compliant - Cookies Set Before Consent", initial_cookies_length, final_cookies_length
        elif cookie_banner:
            return "Partially Compliant - Cookie Banner Present notworking accept/reject ", initial_cookies_length, final_cookies_length
        else:
            return "Non-Compliant - No Consent Mechanism", initial_cookies_length, final_cookies_length

    except Exception as e:
        logging.error(f"Error in cookie consent evaluation: {e}")
        return "Error in Cookie Consent Evaluation", 0, 0  # Return three values even in case of error

# Granular consent testing
def evaluate_granular_consent(driver):
    try:
        preferences_button = driver.find_elements(By.XPATH, 
            "//*[contains(text(), 'Preferences') or contains(text(), 'preferences') or contains(text(), 'Settings') or contains(text(), 'settings')]"
        )
        if preferences_button:
            try:
                WebDriverWait(driver, 3).until(EC.element_to_be_clickable(preferences_button[0])).click()
            except Exception:
                driver.execute_script("arguments[0].click();", preferences_button[0])
            time.sleep(2)

            toggle_xpath = (
                "//input[@type='checkbox' and (contains(@id, 'analytics') or contains(@id, 'marketing') "
                "or contains(@value, 'analytics') or contains(@value, 'marketing') "
                "or contains(@aria-label, 'analytics') or contains(@aria-label, 'marketing'))]"
            )
            toggles = driver.find_elements(By.XPATH, toggle_xpath)

            for toggle in toggles:
                try:
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(toggle)).click()
                    time.sleep(1)
                except Exception as e:
                    logging.warning(f"Error interacting with toggle: {e}")

            return "Granular Consent Evaluated"
        else:
            return "No Granular Options Found"
    except Exception as e:
        logging.error(f"Error in granular consent evaluation: {e}")
        return "Error in Granular Consent Evaluation"

# Load URLs from a file
def load_urls(file_path):
    with open(file_path, "r") as file:
        return [line.strip() for line in file.readlines() if line.strip()]

# Save results to a CSV file
def save_results(results, output_file):
    columns = ["Website", "Compliance Status", "Granular Consent Status", "Cookies Before", "Cookies After"]
    results_df = pd.DataFrame(results, columns=columns)
    results_df.to_csv(output_file, index=False)

# Main function
def main():
    
    input_file = "E:/uni/1fifth1/s/mo3a/project/24-1/data/input/1.csv"  # Input CSV file
    output_file = "E:/uni/1fifth1/s/mo3a/project/24-1/data/output/1out.csv"  # Output CSV file

    urls = load_urls(input_file)
    driver = setup_driver()

    results = []
    for index, url in enumerate(urls, start=1):
        logging.info(f"Checking: {url}")
        print(f"[{index}/{len(urls)}]")
        for protocol in ["https://", "http://"]:
            full_url = protocol + url
            logging.info(f"Trying: {full_url}")

            compliance_status, initial_cookies_length, final_cookies_length = evaluate_cookie_consent(driver, full_url)
            granular_status = evaluate_granular_consent(driver)

            #cookies_before = len(analyze_cookies_with_js(driver))
            #cookies_after = len(analyze_cookies_with_js(driver))

            results.append([full_url, compliance_status, granular_status, initial_cookies_length, final_cookies_length])
            if not compliance_status.startswith("Error"):
                break
        
        save_results(results, output_file)

    driver.quit()
    save_results(results, output_file)
    logging.info(f"Results saved to {output_file}")

if __name__ == "__main__":
    main()


2025-01-24 14:54:35,239 - INFO - ====== WebDriver manager ======
2025-01-24 14:54:36,798 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-24 14:54:37,017 - INFO - Get LATEST chromedriver version for google-chrome
2025-01-24 14:54:37,215 - INFO - Driver [C:\Users\leneo\.wdm\drivers\chromedriver\win64\132.0.6834.110\chromedriver-win32/chromedriver.exe] found in cache
2025-01-24 14:54:40,419 - INFO - Checking: mit.edu
2025-01-24 14:54:40,419 - INFO - Trying: https://mit.edu


[1/200]


2025-01-24 14:54:42,570 - INFO - Checking: harvard.edu
2025-01-24 14:54:42,571 - INFO - Trying: https://harvard.edu


[2/200]


2025-01-24 14:54:54,772 - INFO - Checking: stanford.edu
2025-01-24 14:54:54,774 - INFO - Trying: https://stanford.edu


...

2025-01-24 15:39:23,319 - ERROR - Error in cookie consent evaluation: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00516FB3+25091]
	(No symbol) [0x0049E5D4]
	(No symbol) [0x0037B1C9]
	(No symbol) [0x003B251B]
	(No symbol) [0x003E1D4A]
	(No symbol) [0x003DD831]
	(No symbol) [0x003DCDC4]
	(No symbol) [0x0035313D]
	(No symbol) [0x0035368E]
	(No symbol) [0x00353B1D]
	GetHandleVerifier [0x0080D5B3+3131395]
	GetHandleVerifier [0x0081DDA4+3198964]
	GetHandleVerifier [0x00818CC2+3178258]
	GetHandleVerifier [0x005B3290+664800]
	(No symbol) [0x004A744D]
	(No symbol) [0x00352E10]
	(No symbol) [0x003529C0]
	GetHandleVerifier [0x0088A75C+3643820]
	BaseThreadInitThunk [0x76697BA9+25]
	RtlInitializeExceptionChain [0x7766C0CB+107]
	RtlClearBits [0x7766C04F+191]

2025-01-24 15:39:23,324 - ERROR - Error in granular consent evaluation: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00516FB3+25091]
	(No symbol) [0x0049E5D4]
	(No symbol) [0x0037B1C9]
	(No symbol) [0x003B251B

[196/200]
[197/200]
[198/200]
[199/200]


2025-01-24 15:39:23,578 - INFO - Checking: uidaho.edu
2025-01-24 15:39:23,587 - INFO - Trying: https://uidaho.edu
2025-01-24 15:39:23,593 - ERROR - Error in cookie consent evaluation: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00516FB3+25091]
	(No symbol) [0x0049E5D4]
	(No symbol) [0x0037B1C9]
	(No symbol) [0x003B251B]
	(No symbol) [0x003E1D4A]
	(No symbol) [0x003DD831]
	(No symbol) [0x003DCDC4]
	(No symbol) [0x0035313D]
	(No symbol) [0x0035368E]
	(No symbol) [0x00353B1D]
	GetHandleVerifier [0x0080D5B3+3131395]
	GetHandleVerifier [0x0081DDA4+3198964]
	GetHandleVerifier [0x00818CC2+3178258]
	GetHandleVerifier [0x005B3290+664800]
	(No symbol) [0x004A744D]
	(No symbol) [0x00352E10]
	(No symbol) [0x003529C0]
	GetHandleVerifier [0x0088A75C+3643820]
	BaseThreadInitThunk [0x76697BA9+25]
	RtlInitializeExceptionChain [0x7766C0CB+107]
	RtlClearBits [0x7766C04F+191]

2025-01-24 15:39:23,597 - ERROR - Error in granular consent evaluation: Message: invalid session id
Stacktrace:


[200/200]


2025-01-24 15:39:25,943 - INFO - Results saved to E:/uni/1fifth1/s/mo3a/project/24-1/data/output/1out.csv
